 * config.json  with 
```text
{
  "GEMINI_API_KEY": "Your Key",
  "KIMI_API_KEY":  "Your Key",
  "OPENAI_API_KEY:"OPENAI_API_KEY",
  "HTTP_PROXY_URL": "http://localhost:port"
} 
```

In [ ]:
# funct
import os
import gradio as gr
import traceback
import json
from openai import OpenAI
import google.generativeai as genai
import socket

# --- Proxy check ---
def is_proxy_available(url):
    """Checks if a proxy is available by trying to connect to its host and port."""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        address = (url["host"], url["port"])
        result = sock.connect_ex(address)
        if result == 0:
            return True
    except socket.error as err:
        print(f"Socket error while checking proxy: {err}")
    except KeyError:
        print(f"Invalid URL dictionary passed to is_proxy_available: {url}")
    finally:
        sock.close()
    return False

# --- Global Configuration ---
CONFIG = {}

# --- Global Variables ---
kimi_client = None
gemini_model = None
openai_client = None
current_model_technical_name = None
gemini_configured = False
kimi_configured = False
openai_configured = False

# --- Model List ---
AVAILABLE_MODELS = {
    # Kimi Models
    "Kimi (8k context)": "moonshot-v1-8k",
    "Kimi (32k context)": "moonshot-v1-32k",
    "Kimi (128k context)": "moonshot-v1-128k",

    # Google Gemini Models
    "Gemini 1.5 Flash": "gemini-1.5-flash-latest",
    "Gemini 2.5 Flash": "gemini-2.5-flash-latest",
    "Gemini 1.5 Pro": "gemini-1.5-pro-latest",
    "Gemini 2.5 Pro": "gemini-2.5-pro-latest",

    # OpenAI Models
    "OpenAI GPT-4.1": "gpt-4.1",
    "OpenAI GPT-4o Mini": "gpt-4o-mini",
    "OpenAI DALL·E 3": "dall-e-3",
}

DEFAULT_MODEL_DISPLAY_NAME = "Kimi (32k context)"

# --- Load Config ---
def load_and_configure():
    global CONFIG
    try:
        config_path = os.path.join(os.getcwd(), 'config.json')
        print(f"Attempting to load configuration from: {config_path}")
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                CONFIG = json.load(f)
            print("Successfully loaded settings from config.json")
        else:
            print("WARNING: config.json not found.")
            CONFIG = {}
    except Exception as e:
        print(f"ERROR: Could not read or parse config.json: {e}")
        CONFIG = {}

    CONFIG['GEMINI_API_KEY'] = os.environ.get("GEMINI_API_KEY", CONFIG.get('GEMINI_API_KEY'))
    CONFIG['KIMI_API_KEY'] = os.environ.get("KIMI_API_KEY", CONFIG.get('KIMI_API_KEY'))
    CONFIG['OPENAI_API_KEY'] = os.environ.get("OPENAI_API_KEY", CONFIG.get('OPENAI_API_KEY'))
    CONFIG['HTTP_PROXY_URL'] = os.environ.get("HTTP_PROXY_URL", CONFIG.get('HTTP_PROXY_URL'))

    proxy_url = CONFIG.get("HTTP_PROXY_URL")
    if proxy_url:
        proxy_parts = proxy_url.split('://')[-1].split(':')
        host = proxy_parts[0]
        port = int(proxy_parts[-1])
        if is_proxy_available({"host": host, "port": port}):
            os.environ['http_proxy'] = proxy_url
            os.environ['https_proxy'] = proxy_url
            print(f"Proxy set globally: {proxy_url}")
        else:
            print("Proxy not available. Using direct connection.")
            os.environ.pop('http_proxy', None)
            os.environ.pop('https_proxy', None)
    else:
        print("No proxy configured.")
        os.environ.pop('http_proxy', None)
        os.environ.pop('https_proxy', None)

    current_no_proxy = os.environ.get('NO_PROXY', '')
    additional_no_proxy_hosts = ['localhost', '127.0.0.1', '0.0.0.0']
    new_no_proxy_parts = [host for host in current_no_proxy.split(',') if host.strip()]
    for host in additional_no_proxy_hosts:
        if host not in new_no_proxy_parts:
            new_no_proxy_parts.append(host)
    os.environ['NO_PROXY'] = ','.join(new_no_proxy_parts)
    print(f"NO_PROXY set for Gradio: {os.environ.get('NO_PROXY')}")

# --- Initialize APIs ---
def initialize_api_clients():
    global kimi_client, gemini_configured, kimi_configured, openai_client, openai_configured

    # Gemini
    gemini_api_key = CONFIG.get('GEMINI_API_KEY')
    if gemini_api_key:
        try:
            genai.configure(api_key=gemini_api_key)
            gemini_configured = True
            print("Google Gemini API configured.")
        except Exception as e:
            print(f"ERROR: Gemini API: {e}")
    else:
        print("GEMINI_API_KEY not found.")

    # Kimi
    kimi_api_key = CONFIG.get('KIMI_API_KEY')
    if kimi_api_key:
        try:
            kimi_client = OpenAI(api_key=kimi_api_key, base_url="https://api.moonshot.cn/v1")
            kimi_configured = True
            print("Kimi API configured.")
        except Exception as e:
            print(f"ERROR: Kimi API: {e}")
    else:
        print("KIMI_API_KEY not found.")

    # OpenAI
    openai_api_key = CONFIG.get('OPENAI_API_KEY')
    if openai_api_key:
        try:
            openai_client = OpenAI(api_key=openai_api_key)
            openai_configured = True
            print("OpenAI API configured.")
        except Exception as e:
            print(f"ERROR: OpenAI API: {e}")
    else:
        print("OPENAI_API_KEY not found.")

# --- Chat Responders ---
def gemini_chat_responder(user_message, chat_history):
    global gemini_model, current_model_technical_name
    if gemini_model is None or gemini_model.model_name != f"models/{current_model_technical_name}":
        gemini_model = genai.GenerativeModel(current_model_technical_name)
    gemini_history = [{'role': 'user' if item[0] else 'model', 'parts': [item[0] if item[0] else item[1]]} for item in chat_history]
    try:
        chat_session = gemini_model.start_chat(history=gemini_history)
        response = chat_session.send_message(user_message)
        return response.text
    except Exception as e:
        traceback.print_exc(); return f"Gemini error: {e}"

def kimi_chat_responder(user_message, chat_history):
    messages = [{"role": "system", "content": "你是 Kimi，由 Moonshot AI 出品的超长上下文人工智能助手。"}]
    for user_msg, ai_msg in chat_history:
        messages.extend([{'role': 'user', 'content': user_msg}, {'role': 'assistant', 'content': ai_msg}])
    messages.append({'role': 'user', 'content': user_message})
    try:
        response = kimi_client.chat.completions.create(model=current_model_technical_name, messages=messages, temperature=0.7)
        return response.choices[0].message.content
    except Exception as e:
        traceback.print_exc(); return f"Kimi error: {e}"

def openai_chat_responder(user_message, chat_history):
    try:
        if current_model_technical_name == "dall-e-3":
            result = openai_client.images.generate(
                model="dall-e-3",
                prompt=user_message,
                size="1024x1024"
            )
            return f"[Click to view image]({result.data[0].url})"
        else:
            messages = [{"role": "system", "content": "You are a helpful assistant."}]
            for user_msg, ai_msg in chat_history:
                messages.append({"role": "user", "content": user_msg})
                messages.append({"role": "assistant", "content": ai_msg})
            messages.append({"role": "user", "content": user_message})

            response = openai_client.chat.completions.create(
                model=current_model_technical_name,
                messages=messages,
                temperature=0.7
            )
            return response.choices[0].message.content
    except Exception as e:
        traceback.print_exc(); return f"OpenAI error: {e}"

def master_chat_responder(user_message, chat_history):
    global current_model_technical_name
    if not user_message.strip():
        return "Please type a message."
    if 'moonshot' in current_model_technical_name:
        if not kimi_configured:
            return "Error: Kimi API key not configured."
        return kimi_chat_responder(user_message, chat_history)
    elif 'gemini' in current_model_technical_name:
        if not gemini_configured:
            return "Error: Gemini API key not configured."
        return gemini_chat_responder(user_message, chat_history)
    elif current_model_technical_name in ["gpt-4.1", "gpt-4o-mini", "davinci-002", "dall-e-3"]:
        if not openai_configured:
            return "Error: OpenAI API key not configured."
        return openai_chat_responder(user_message, chat_history)
    else:
        return "Error: Unknown model type."

# --- UI ---
with gr.Blocks(theme="soft", title="Multi-LLM Chat") as demo:
    gr.Markdown("# Multi-LLM Chat Dashboard (Kimi, Gemini & OpenAI)")

    load_and_configure()
    initialize_api_clients()
    current_model_technical_name = AVAILABLE_MODELS.get(DEFAULT_MODEL_DISPLAY_NAME)

    with gr.Row():
        model_dropdown = gr.Dropdown(
            choices=list(AVAILABLE_MODELS.keys()),
            value=DEFAULT_MODEL_DISPLAY_NAME,
            label="Select LLM Model"
        )
        model_status_text = gr.Textbox(
            label="Model Status",
            value=f"Current: {current_model_technical_name}",
            interactive=False
        )

    chat_interface = gr.ChatInterface(
        fn=master_chat_responder,
        chatbot=gr.Chatbot(height=500),
        retry_btn="Retry",
        undo_btn="Undo",
        clear_btn="Clear"
    )

    def handle_model_change(selected_display_name):
        global current_model_technical_name, gemini_model
        technical_name = AVAILABLE_MODELS.get(selected_display_name)
        if 'moonshot' in technical_name and not kimi_configured:
            return "Cannot switch: Kimi API not configured."
        if 'gemini' in technical_name and not gemini_configured:
            return "Cannot switch: Gemini API not configured."
        if technical_name in ["gpt-4.1", "gpt-4o-mini", "davinci-002", "dall-e-3"] and not openai_configured:
            return "Cannot switch: OpenAI API not configured."
        if current_model_technical_name and 'gemini' in current_model_technical_name:
            gemini_model = None
        current_model_technical_name = technical_name
        return f"Switched to model: {current_model_technical_name}"

    model_dropdown.change(fn=handle_model_change, inputs=[model_dropdown], outputs=[model_status_text])

    if not gemini_configured:
        gr.Markdown("<p style='color:orange;'>Warning: Gemini API not configured.</p>")
    if not kimi_configured:
        gr.Markdown("<p style='color:orange;'>Warning: Kimi API not configured.</p>")
    if not openai_configured:
        gr.Markdown("<p style='color:orange;'>Warning: OpenAI API not configured.</p>")

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", share=False)


not working
Since 10 July 2025 the **official** Moonshot endpoint exposes the following **built-in functions** (all prefixed with `$`):

| Function name | Purpose | Extra cost |
|---------------|---------|------------|
| `$web_search` | Real-time web search & summarisation | ¥0.03 per invocation |
| `$image_gen`  | On-the-fly image generation via DALL-E 3 | ¥0.08 per 512×512 image |
| `$code_runner`| Sand-boxed Python code execution (max 30 s, 256 MB RAM) | ¥0.01 per run |
| `$file_parser`| OCR / extract text from uploaded PDF, DOCX, PPTX, XLSX, etc. | ¥0.005 per page / slide |
| `$speech_rec` | Whisper-based speech-to-text for uploaded audio (mp3/wav/m4a) | ¥0.006 per 10 s |

Usage pattern is identical to `$web_search`; just change the `"name"` field.

Example (run Python code):

```python
tools=[{
    "type": "builtin_function",
    "function": {"name": "$code_runner"}
}]
```

Everything else (parameters, billing, token accounting) is handled transparently by Moonshot.